# NDPointIndex

Needs latest xarray '2025.7.1'

In [ ]:
import numpy as np
import wradlib as wrl
import xarray as xr
from open_radar_data import DATASETS

In [ ]:
xr.__version__

## Load radar data + georeference + NDPointIndex

In [ ]:
fname = DATASETS.fetch("DWD-Vol-2_99999_20180601054047_00.h5")
ds = xr.open_dataset(fname, engine="gamic", group="sweep_9").xradar.georeference()
ds = ds.set_xindex(("x", "y"), xr.indexes.NDPointIndex)

In [ ]:
ds.DBZH.plot(x="x", y="y", xlim=(-10e3, 10e3), ylim=(-10e3, 10e3))

## Selection + coordinate assignment -> nearest neighbour interpolation

1. Create 1D DataArrays for x and y selection
2. use .sel with above 1D DataArrays
3. assign those DataArrays to selection

In [ ]:
%%time
y = xr.DataArray(np.arange(-100e3, 100e3, 500), dims="y", name="y", attrs=ds.y.attrs)
x = xr.DataArray(np.arange(-100e3, 100e3, 500), dims="x", name="x", attrs=ds.x.attrs)

actual = ds.sel(
    y=y,
    x=x,
    method="nearest",
)

actual = actual.assign(
    x=x,
    y=y,
)

In [ ]:
display(actual)

In [ ]:
actual.DBZH.plot(xlim=(-10e3, 10e3), ylim=(-10e3, 10e3))

## wradlib nearest neighbour

Simple wradlib approach, not fixed for performance.

In [ ]:
%%time
xgrid = np.arange(-100e3, 100e3, 500)
ygrid = np.arange(-100e3, 100e3, 500)
cart = xr.Dataset(coords={"x": x, "y": y})
with xr.set_options(keep_attrs=True):
    gridded = ds.wrl.comp.togrid(
        cart, radius=100000.0, center=(0, 0), interpol=wrl.ipol.Nearest
    )

In [ ]:
display(gridded)

In [ ]:
gridded.DBZH.plot(xlim=(-10e3, 10e3), ylim=(-10e3, 10e3))

## Check for equality

In [ ]:
assert (gridded.DBZH - actual.DBZH).sum() == 0

## Plot

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
actual.DBZH.plot(ax=ax1, xlim=(-10e3, 10e3), ylim=(-10e3, 10e3), cmap="HomeyerRainbow")
gridded.DBZH.plot(ax=ax2, xlim=(-10e3, 10e3), ylim=(-10e3, 10e3), cmap="HomeyerRainbow")